In [ ]:
#from keras.models import load_model
#import imutils # Package of convenience functions
#from skimage import io
#import matplotlib.pyplot as plt
#import requests
#import timeit

In [ ]:
# #############################################################################
# Importing necessary libraries for creating model
from keras.models import model_from_json
import os
import pandas as pd
import numpy as np
import keras
import cv2 # importing open CV library
import dlib   # For frontal face identity
from PIL import Image
import csv
import face_recognition # importing face recognition library
from sklearn.metrics import classification_report, confusion_matrix
import sys
from zipfile import ZipFile

In [ ]:
# #############################################################################
# Unpack and extract images paths

# Create a ZipFile Object and load sample.zip in it
with ZipFile('../../OUI/OUI.zip', 'r') as zipObj:
   # Extract all the contents of zip file 
   zipObj.extractall('../OUI_unarchive')

images = []
if len(next(os.walk('../OUI_unarchive/OUI/'))[1]) == 0: #case when images are in one folder
    for image in os.listdir('../OUI_unarchive/OUI/'): 
        if image[-3:] == 'jpg' or image[-3:] == 'png':
            images.append('../OUI_unarchive/OUI/' + image)
else: #case when images are in several folders
    for path in ['../OUI_unarchive/OUI/' + next(os.walk('../OUI_unarchive/OUI/'))[1][n] for n in range(len(next(os.walk('../OUI_unarchive/OUI/'))[1]))]:
        for image in os.listdir(path+'/'): 
            if image[-3:] == 'jpg' or image[-3:] == 'png':
                images.append(path+'/' + image)
            
print('Total number of images:', len(images))

In [ ]:
images = ['female1.jpg', 'female2.jpg', 'female3.jpg', 'male1.jpg', 'male2.jpg', 'male3.jpg']

In [ ]:

# #############################################################################
# loading the trained model and weights
json_file = open('../Trained Model/model.json', 'r')     # Loading the jason file
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)
loaded_model.load_weights("../Trained Model/model.h5")    # Loading the weights


def detect_faces(image):                    # Create a face detector
    face_detector = dlib.get_frontal_face_detector()
    detected_faces = face_detector(image, 1)    # Run detector and get bounding boxes of the faces on image
    face_frames = [(x.left(), x.top(),
                    x.right(), x.bottom()) for x in detected_faces]  # To identify the frame size
    return face_frames


def resize_im(image, hight, width):          # Resizing the image
    resized_im = image.resize((hight, width))
    return resized_im


def reshape(numpy_array):            # Reshape the image
    d = np.stack(numpy_array)
    reshaped = d.reshape(1, d.size)
    return reshaped


def GIGIDL(image_path):
    f = open("../storage.csv", "w")
    img= cv2.imread(image_path)
    detected_faces = detect_faces(img)
    for n, face_rect in enumerate(detected_faces):
        fa = Image.fromarray(img).crop(face_rect)   # Cropping the face or faces
        s = np.array(resize_im(fa, 100,100))
        c = cv2.cvtColor(s, cv2.COLOR_BGR2GRAY)    # Converting the image
        re_data = reshape(c)
        np.savetxt(f, re_data, delimiter=",", fmt = '%d')  # Save the converted image data
        cv2.waitKey(0)
        cv2.destroyAllWindows()    # Need to kill the image window
    #reading the faces from the temp file
    data = pd.read_csv('../storage.csv', header = None)
    df_x = data.iloc[:,0:].values.reshape(len(data),100,100,1)   # Creating the requird formate
    f_x = np.array(df_x)  # Need to convert as a array
    loaded_model.compile(loss='categorical_crossentropy',optimizer = 'adadelta', metrics = ['accuracy'])
    prob = np.asmatrix(loaded_model.predict(f_x))
    if prob[0,0] > prob[0,1]:
             gender= 'f'
    if prob[0,0] < prob[0,1]:
             gender='m'
    return gender
    



In [ ]:
with open('OUI.csv', 'w', newline='') as myfile:             # Here output written in the file
    wr = csv.writer(myfile,quoting=csv.QUOTE_ALL)
    wr.writerow(['Image_name','Pred_Gender'])

    for imagelocation in images:                        # Get the images location
            imageInfo = imagelocation.split('/')
            image =imageInfo[-1].split('.')
            #imageName = imageInfo[-2]+'/'+'.'.join(image[-3:])
            imageName = imagelocation
            try:
                GIGIDL_gender=GIGIDL(imagelocation)   # return the predicted gender
                print(imageName, GIGIDL_gender)
                wr = csv.writer(myfile,quoting=csv.QUOTE_ALL)
                wr.writerow([imageName,GIGIDL_gender])
            except:
                wr = csv.writer(myfile,quoting=csv.QUOTE_ALL)
                wr.writerow([imageName,'u'])   #if there is an error in returning gender , write undefined
                print("Oops!",sys.exc_info()[0],"occured.")
                print()

In [ ]:
def annotation_ext(input_dir,output_dir):
    # Create a ZipFile Object and load sample.zip in it
    with ZipFile(input_dir, 'r') as zipObj:
    # Extract all the contents of zip file 
        zipObj.extractall(output_dir)
    files=[output_dir+'/'+name for name in os.listdir(output_dir) if os.path.isfile(os.path.join(output_dir, name))]
    df = pd.DataFrame()
    for file in files:
        data = pd.read_csv(file, sep="\t")  #load each file as dataframe
        df= df.append(data, ignore_index = True)
    df= df.astype(str)
    df['Image_name']=df['user_id']+'/'+ df['face_id']+'.'+df['original_image'] # make Image name similar to the one in the image path
    df= df[['Image_name','gender']] # keep only gender and imagename
    return df.drop_duplicates()

In [ ]:
result= annotation_ext('../../OUI/OUI_annotations.zip','../OUI_annotations')
result

In [ ]:
### Precision, recall, F1_score
#read the result_file
df_pred=pd.read_csv('/Users/nadabeili/Desktop/Improved-Gender-Identifier/OUI.csv')
result_data= pd.merge(result,df_pred, how= 'inner', on='Image_name')
result_data.to_csv('../all_result.csv')
result_data.dropna(inplace=True)
# True values
y_true = result_data['gender']
# Predicted values
y_pred = result_data['Pred_Gender']

# Print the confusion matrix
print(confusion_matrix(y_true, y_pred))
# Print the precision and recall, among other metrics
print(classification_report(y_true, y_pred, digits=4))

#printing the metrics
metrics_dict=classification_report(y_true, y_pred,output_dict=True)

#precision:
print('Precision:',round(metrics_dict['weighted avg']['precision'],4))
#Recall
print('Recall:',round(metrics_dict['weighted avg']['recall'],4))
#F1-score
print('F1-score:',round(metrics_dict['weighted avg']['f1-score'],4))
#accuracy
print('Accuracy:',round(metrics_dict['accuracy'],4))

In [ ]:
result_data